In [1]:
#python 2 and 3 comptibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import json
import io
import gzip
import base64
from astropy.io import fits
import matplotlib.pylab as plt
import pickle as pkl
import numpy as np
import sys 
import os
%matplotlib inline

PATH_TO_PROJECT = os.path.abspath(
    os.path.join(os.path.dirname('__file__'), ''))
sys.path.append(PATH_TO_PROJECT)
sys.path.append(os.path.join(PATH_TO_PROJECT,'..'))

data_path = '../AlerceDHtest/datasets/ZTF'

path = data_path+'/broker_reals.json'
with open(path, "r") as f:
        dataset = json.load(f)

In [2]:
#print object ids
#for alert in dataset["query_result"]:
#    print(alert["objectId"])

In [3]:
#build dict by alert id
alerts = {}
passbands = ["g", "r", "i"]
for alert in dataset["query_result"]:
    dict_in_stamp = {'science_stamp':None, 'filter': None, 'fwhm': None, 'mjd': None, 'field': None, 'object_id': None}
    #get params to save in dict
    dict_in_stamp['filter'] = passbands[alert["candidate"]["fid"]-1]
    dict_in_stamp['fwhm'] = alert["candidate"]["fwhm"]
    dict_in_stamp['mjd'] = alert["candidate"]["jd"]
    dict_in_stamp['object_id'] = alert['objectId']
    dict_in_stamp['field'] = alert["candidate"]["field"]
    dict_in_stamp['_id'] = alert["_id"]
    #other observation conditions
    dict_in_stamp['exp_time'] = 30#alert["candidate"]["exptime"]
    dict_in_stamp['limmag3'] = None
    dict_in_stamp['limmag5'] = alert["candidate"]["diffmaglim"]
    dict_in_stamp['obs_days'] = alert["candidate"]["jd"]
    dict_in_stamp['seeing'] = alert["candidate"]["fwhm"]
    dict_in_stamp['sky_brightness'] = None #Needs to be calculated from science stamp
    dict_in_stamp['zero_point'] =  1.0 #We dont have it but a constant is enough
    #stamps
    stamp = alert['cutoutScience']['stampData']
    stamp = base64.b64decode(stamp["$binary"].encode())
    with gzip.open(io.BytesIO(stamp), 'rb') as f:
        with fits.open(io.BytesIO(f.read())) as hdul:
            img = hdul[0].data
    dict_in_stamp['science_stamp'] = img
    
    stamp_id = alert["_id"]
    
    if stamp_id in alerts.keys():
        print(stamp_id)
        continue
    else:
        alerts[stamp_id] = dict_in_stamp

In [4]:
def createCircularMask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])
    radius = np.round(radius)
    
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [5]:
# add masked sky and sky array to dicts
n_sigmas = 4

#check if values are already in dict, to avoid second innecesarry runs
if 'masked_sky' not in alerts[list(alerts.keys())[0]].keys():
    for alert_id in alerts:
        alert = alerts[alert_id] 
        fw = alert['fwhm']
        mask = ~createCircularMask(alert['science_stamp'].shape[0], alert['science_stamp'].shape[1], radius=n_sigmas*(fw/2.35482))
        masked_sky = alert['science_stamp']*mask
        alert['masked_sky'] = masked_sky

        sky_pixels_cord = np.argwhere(mask==True)
        sky_pixels = np.array([masked_sky[cord[0],cord[1]] for cord in list(sky_pixels_cord)])
        alert['sky_pixel_values'] = sky_pixels

In [6]:
#class to filter sky without considering nans
class Sky_filter:

    def __init__(self, sky_pixels, n_sigma=2, filter_iterations=5):
        
        self.n = n_sigma
        self.sigmas = np.empty((0))
        self.n_sigmas = np.empty((0))
        self.sky_pixels = sky_pixels[np.logical_not(np.isnan(sky_pixels))]      
        
    def get_n_sigma(self, sky_pixels):
        sigma = np.std(sky_pixels)
        #print(np.mean(sky_pixels))
        #print(np.mean(sky_pixels)+sigma)
        n_sigma = np.mean(sky_pixels)+self.n*sigma
        self.sigmas = np.append(self.sigmas,np.mean(sky_pixels)+sigma)
        self.n_sigmas = np.append(self.n_sigmas,n_sigma)
        return n_sigma
    
    def filter_sky_once(self):
        last_filtered_sky = self.filtered_sky['it%i'%(len(list(self.filtered_sky.keys()))-1)]
        n_sigma = self.get_n_sigma(last_filtered_sky)
        pixels_over_sigma = np.abs(last_filtered_sky)>n_sigma
        pixels_over_std_cord = np.argwhere(pixels_over_sigma==True)
        #print(pixels_over_std_cord)
        new_filtered_sky = np.delete(last_filtered_sky, pixels_over_std_cord)
        self.filtered_sky['it%i'%len(list(self.filtered_sky.keys()))] = new_filtered_sky
        
    def iterative_filtering(self, iterations):
        #print(self.sky_pixels.shape)
        if np.isnan(self.sky_pixels).any():
            return None
            
        self.sigmas = np.empty((0))
        self.n_sigmas = np.empty((0))
        self.filtered_sky = {'it0':self.sky_pixels}
        for i in range(iterations):
            self.filter_sky_once()
        return self.filtered_sky

    def check_availability(self, array, idx):
        array = np.array(array)
        if array.shape[0]-1<idx:
            check_nans = self.iterative_filtering(idx+1)
        if check_nans is None:
            raise ValueError('nans in sky pixels')
            
    def plot_array(self, array):
        array = np.array(array)
        x = np.arange(0, array.shape[0])
        plt.plot(x,array)
            
    def plot_filtered_sky_sigmas(self, idx=0, x_pos=32, y_pos=32):
        self.check_availability(self.n_sigmas, idx)
        
        #plt.figure(figsize=(12,12))
        line = np.arange(0,self.sky_pixels.shape[0])
        n_sigma_line = np.full(self.sky_pixels.shape[0], self.n_sigmas[idx])
        mean = np.full(self.sky_pixels.shape[0], np.mean(self.sky_pixels))
        plt.scatter(line, self.sky_pixels)
        #plt.lim(np.min(self.sky_pixels),np.max(self.sky_pixels))
        plt.plot(line, n_sigma_line, color='black', label='sigma')
        plt.plot(line, mean, '--', color='black', label='mean')
        #print(self.sky_pixels)
        #print(self.sky_pixels.shape)
        plt.title(r'non-filtered $\sigma$[%f], max: %f'%(self.n_sigmas[idx].round(decimals=3), np.max(self.sky_pixels)))
        plt.legend()
        plt.show()

        filtered_sky = self.filtered_sky['it%i'%idx]
        line = np.arange(0,filtered_sky.shape[0])
        n_sigma_line = np.full(filtered_sky.shape[0], self.n_sigmas[idx])
        mean = np.full(filtered_sky.shape[0], np.mean(filtered_sky))
        plt.scatter(line, filtered_sky)
        plt.plot(line, n_sigma_line, color='black', label='sigma')
        plt.plot(line, mean, '--', color='black', label='mean')
        plt.title(r'filtered $\sigma$['+str(idx)+r']: '+str(self.n_sigmas[idx].round(decimals=3)))
        plt.legend()
        plt.show()
        
        self.plot_array(self.n_sigmas)
        plt.title(r'$\sigma$´s progretions')
        plt.show()

In [7]:
# add filtered sky (after filter_iterations iterations) to dict
filter_iterations = 10

#check if values are already in dict, to avoid second innecesarry runs
if 'filtered_sky_pixel_values' not in alerts[list(alerts.keys())[0]].keys():
    #i=0
    for alert_id in alerts:
        #print(i)
        #i+=1
        alert = alerts[alert_id] 
        sky_values = alert['sky_pixel_values']
        sky_filter = Sky_filter(sky_values)
        filtered_sky = sky_filter.iterative_filtering(iterations=filter_iterations)
        #get last iteration filtered sky
        last_iteration_filtered_sky = filtered_sky[list(filtered_sky.keys())[-1]]
        mean_sky_value = np.mean(last_iteration_filtered_sky)
        #add data to dict
        alert['filtered_sky_pixel_values'] = last_iteration_filtered_sky
        alert['sky_value'] = mean_sky_value

In [8]:
alerts[alert_id].keys()

dict_keys(['science_stamp', 'filter', 'fwhm', 'mjd', 'field', 'object_id', '_id', 'exp_time', 'limmag3', 'limmag5', 'obs_days', 'seeing', 'sky_brightness', 'zero_point', 'masked_sky', 'sky_pixel_values', 'filtered_sky_pixel_values', 'sky_value'])

# Filter alerts that have less than N_ALERTS_IN_MJD

In [9]:
#all mjds
all_mjd = [alerts[alert_id]['mjd'] for alert_id in alerts]

In [10]:
#count mjds and generate dict mjd/count_mjd
#TODO: add frequency to dict
from collections import Counter
mjd_group_dict = Counter(all_mjd)
#print(Counter(all_mjd).keys()) # equals to list(set(words))
#print(Counter(all_mjd).values()) # counts the elements' frequency

In [11]:
#filter alerts that have less than 10 mjd freq
N_ALERTS_IN_MJD = 20
filtered_alerts = {}

for alert_id in alerts:
    alert_mjd = alerts[alert_id]['mjd']
    if mjd_group_dict[alert_mjd]>=N_ALERTS_IN_MJD:
        filtered_alerts[alert_id] = alerts[alert_id]
        

In [17]:
filtered_alerts[alert_id].keys()

dict_keys(['science_stamp', 'filter', 'fwhm', 'mjd', 'field', 'object_id', '_id', 'exp_time', 'limmag3', 'limmag5', 'obs_days', 'seeing', 'sky_brightness', 'zero_point', 'masked_sky', 'sky_pixel_values', 'filtered_sky_pixel_values', 'sky_value'])

# Group alerts by field (every object is considered as a diferrent field)

In [12]:
all_objectsId = [filtered_alerts[alert_id]['object_id'] for alert_id in filtered_alerts]
objectId_uniques = np.unique(all_objectsId)

In [13]:
#name field
field_objectId_dict = {}
for i, objectId in enumerate(objectId_uniques):
    field_objectId_dict['Field%s' % str(i+1).zfill(2)] = objectId

In [14]:
#create dict of fields as in https://github.com/rodrigcd/HiTS_simulations
camera_obs_cond = {}
camera_obs_cond["obs_conditions"] = {}
for field in field_objectId_dict.keys():
    camera_obs_cond["obs_conditions"][field] = []
    for alert_id in filtered_alerts:
        if filtered_alerts[alert_id]['object_id'] == field_objectId_dict[field]:
            camera_obs_cond["obs_conditions"][field].append(filtered_alerts[alert_id])

In [15]:
camera_obs_cond["obs_conditions"]

{'Field01': [{'science_stamp': array([[123.20407 , 127.22096 , 117.12474 , ..., 128.8524  , 111.45215 ,
           125.553665],
          [122.64505 , 115.096375, 121.96744 , ..., 116.10672 , 119.21034 ,
           125.684555],
          [125.267944, 119.76524 , 122.513824, ..., 129.42937 , 119.66519 ,
           120.97003 ],
          ...,
          [138.09692 , 153.87723 , 139.80548 , ..., 115.56909 , 115.763115,
           126.22919 ],
          [134.58777 , 125.77505 , 120.116066, ..., 115.15088 , 119.28728 ,
           118.001854],
          [120.917816, 131.72041 , 120.86147 , ..., 123.81089 , 118.3566  ,
           128.24765 ]], dtype=float32),
   'filter': 'g',
   'fwhm': 2.609999895095825,
   'mjd': 2458247.8927778,
   'field': 639,
   'object_id': 'ZTF18aarwvfe',
   '_id': '493392771315015067_ZTF18aarwvfe',
   'exp_time': 30,
   'limmag3': None,
   'limmag5': 20.87923812866211,
   'obs_days': 2458247.8927778,
   'seeing': 2.609999895095825,
   'sky_brightness': None,
   'zero

108